### Entrenamiento de Modelos

En todos los pasos, imprimir los resultados sobre el conjunto de entrenamiento y sobre el conjunto de test. Usar siempre GridSearchCV (con kfold) para encontrar los mejores parámetros.

In [1]:
import numpy as np
import pandas as pd

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb
from sklearn.neural_network import MLPClassifier

## Cantidad de procesadores a ser usados
n_jobs = 4

In [2]:
data = pd.read_csv('../datasets/sneep-unificado-2002-2017-CURADO.csv')

/home/nahuel/anaconda3/envs/diploDatos/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (34,36,38,40) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Selección de atributos 

Comenzaremos usando el siguiente listado de atributos:

In [3]:
X = data[['delito1_id','provincia_id','genero_id','nacionalidad_id',
         'es_reincidente_id','anio_condenado','edad_al_ser_condenado',
          'nivel_instruccion_id','estado_civil_id']].astype(int)
y = data['duracion_condena_rango'].astype(int)

xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size = 0.10, random_state = 0)

#### Tareas a realizar:

1- Crear una función que permita almacenar en un Dataframe de Pandas los siguientes datos:
"id_ejecucion", "tipo_modelo", "descripcion", "accuracy", "precision", "recall". Para cada predicción sobre el modelo de test que realicemos cargar los resultados en este df, con el objetivo de tener un log de trabajo. Al finalizar guardar el df en un csv.

In [4]:
# Create the pandas DataFrame 
log = pd.DataFrame(columns = ["id_ejecución", "tipo_modelo", "descripcion", "accuracy", "precision", "recall"]) 
# Save it to .csv
log.to_csv("log.csv")

In [5]:
def save(num_ejec, y_pred, y_true=yTest, tipo_modelo=None, descripcion=None, df=log):
    """
    Función que guarda los datos del modelo
    """
    
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    
    df.loc[num_ejec] = ['ejec' + str(num_ejec)] + [tipo_modelo] + [descripcion] + [accuracy] + [precision] + [recall]
    
    df.to_csv("log.csv")
    
    print("Reporte para el modelo número " + str(num_ejec) + " :")
    print(classification_report(y_true, y_pred))
    print("Actualización de la tabla:")
    print(df)

También definimos una función para hacer la búsqueda de los mejores parámetros:

In [6]:
def grid(model, params, n_jobs=None, verbose=0):

    clf = GridSearchCV(model, params, cv=StratifiedKFold(n_splits=3).split(xTrain, yTrain), 
                       n_jobs=n_jobs, verbose=verbose)
    clf.fit(xTrain, yTrain)

    print("Mejor conjunto de parámetros:")
    print(clf.best_params_, end="\n\n")

    print("Puntajes de la grilla:", end="\n\n")
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("Exactitud: %0.4f (+/-%0.04f) para los parámetros %r" % (np.sqrt(mean), np.sqrt(std), params))
        
    return clf

2- Entrenar un modelo con LogisticRegression

In [7]:
i = 0 # En esta variable guardamos la cantidad de modelos ejecutados hasta el momento

In [8]:
# Regresión Logística
model = LogisticRegression(
    random_state=0,     ## Especifico semilla para reproducibilidad
    solver='liblinear', ## Especifico solver para evitar FutureWarning (sklearn.__version__ = 0.21.3)
    multi_class='auto'  ## Especifico multi_class para evitar FutureWarning (sklearn.__version__ = 0.21.3)
) 
model.fit(xTrain, yTrain)

yPred = model.predict(xTest)

desc = 'default parameters (penalty=l2, C=1.0, random_state=0)'

save(num_ejec=i, y_pred=yPred, tipo_modelo='LogisticRegression', descripcion=desc) 
i+=1

/home/nahuel/anaconda3/envs/diploDatos/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Reporte para el modelo número 0 :
              precision    recall  f1-score   support

           1       0.00      0.00      0.00       288
           2       0.47      0.84      0.60      2012
           3       0.26      0.23      0.24      1658
           4       0.28      0.09      0.14      1143
           5       0.00      0.00      0.00       561
           6       0.00      0.00      0.00       285
           7       0.33      0.52      0.41       929

    accuracy                           0.39      6876
   macro avg       0.19      0.24      0.20      6876
weighted avg       0.29      0.39      0.31      6876

Actualización de la tabla:
  id_ejecución         tipo_modelo  \
0        ejec0  LogisticRegression   

                                         descripcion  accuracy  precision  \
0  default parameters (penalty=l2, C=1.0, random_...  0.386271   0.291249   

     recall  
0  0.386271  


/home/nahuel/anaconda3/envs/diploDatos/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Buscamos mejores parámetros:

In [9]:
logi_para = {
    'penalty': ['l1','l2'],   ## l1 lasso l2 ridge
    'C': np.logspace(-3,3,7),
    'random_state': [0],      ## Especifico semilla para reproducibilidad
    'solver': ['liblinear'],  ## Especifico solver para evitar FutureWarning (sklearn.__version__ = 0.21.3)
    'multi_class': ['auto']   ## Especifico multi_class para evitar FutureWarning (sklearn.__version__ = 0.21.3)
}

model = grid(model = LogisticRegression(), params = logi_para, n_jobs=n_jobs)

Mejor conjunto de parámetros:
{'C': 1.0, 'multi_class': 'auto', 'penalty': 'l2', 'random_state': 0, 'solver': 'liblinear'}

Puntajes de la grilla:

Exactitud: 0.6242 (+/-0.0646) para los parámetros {'C': 0.001, 'multi_class': 'auto', 'penalty': 'l1', 'random_state': 0, 'solver': 'liblinear'}
Exactitud: 0.6239 (+/-0.0641) para los parámetros {'C': 0.001, 'multi_class': 'auto', 'penalty': 'l2', 'random_state': 0, 'solver': 'liblinear'}
Exactitud: 0.6242 (+/-0.0604) para los parámetros {'C': 0.01, 'multi_class': 'auto', 'penalty': 'l1', 'random_state': 0, 'solver': 'liblinear'}
Exactitud: 0.6240 (+/-0.0594) para los parámetros {'C': 0.01, 'multi_class': 'auto', 'penalty': 'l2', 'random_state': 0, 'solver': 'liblinear'}
Exactitud: 0.6240 (+/-0.0609) para los parámetros {'C': 0.1, 'multi_class': 'auto', 'penalty': 'l1', 'random_state': 0, 'solver': 'liblinear'}
Exactitud: 0.6242 (+/-0.0599) para los parámetros {'C': 0.1, 'multi_class': 'auto', 'penalty': 'l2', 'random_state': 0, 'solver': '

Luego corremos el modelo con los mejores parámetros encontrados:

In [25]:
best_params = model.best_params_

model = LogisticRegression(
    solver=best_params['solver'],
    multi_class=best_params['multi_class'],
    penalty=best_params['penalty'],
    C=best_params['C'],
    random_state=best_params['random_state']
)
model.fit(xTrain, yTrain)

yPred = model.predict(xTest)

desc = "penalty={}, C={}, random_state={}"
desc = desc.format(str(best_params['penalty']),
                str(best_params['C']), 
                str(best_params['random_state'])
                )

save(num_ejec=i, y_pred=yPred, tipo_modelo='LogisticRegression', descripcion=desc)
i+=1

Reporte para el modelo número 1 :
              precision    recall  f1-score   support

           1       0.00      0.00      0.00       288
           2       0.47      0.84      0.60      2012
           3       0.26      0.23      0.24      1658
           4       0.28      0.09      0.14      1143
           5       0.00      0.00      0.00       561
           6       0.00      0.00      0.00       285
           7       0.33      0.52      0.41       929

    accuracy                           0.39      6876
   macro avg       0.19      0.24      0.20      6876
weighted avg       0.29      0.39      0.31      6876

Actualización de la tabla:
  id_ejecución         tipo_modelo  \
0        ejec0  LogisticRegression   
1        ejec1  LogisticRegression   

                                         descripcion  accuracy  precision  \
0  default parameters (penalty=l2, C=1.0, random_...  0.386271   0.291249   
1                  penalty=l2, C=1.0, random_state=0  0.386271   0.291249

/home/nahuel/anaconda3/envs/diploDatos/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/nahuel/anaconda3/envs/diploDatos/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


3- Entrenar un modelo de DecisionTreeClassifier

In [26]:
# Árbol de decision
model = DecisionTreeClassifier(random_state=0)
model.fit(xTrain, yTrain)

yPred = model.predict(xTest)

desc = 'default parameters (criterion=gini, max_depth=None, max_features=None, random_state=0)'

save(num_ejec=i, y_pred=yPred, tipo_modelo='DecisionTreeClassifier', descripcion=desc)
i+=1

Reporte para el modelo número 2 :
              precision    recall  f1-score   support

           1       0.20      0.20      0.20       288
           2       0.56      0.65      0.60      2012
           3       0.47      0.45      0.46      1658
           4       0.55      0.49      0.52      1143
           5       0.47      0.44      0.46       561
           6       0.47      0.47      0.47       285
           7       0.64      0.61      0.62       929

    accuracy                           0.52      6876
   macro avg       0.48      0.47      0.47      6876
weighted avg       0.52      0.52      0.52      6876

Actualización de la tabla:
  id_ejecución             tipo_modelo  \
0        ejec0      LogisticRegression   
1        ejec1      LogisticRegression   
2        ejec2  DecisionTreeClassifier   

                                         descripcion  accuracy  precision  \
0  default parameters (penalty=l2, C=1.0, random_...  0.386271   0.291249   
1                  

Buscamos mejores parámetros:

In [28]:
tree_para = {'criterion':['gini','entropy'],
             'max_depth':[None, 4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 20, 30, 40, 50, 70, 90, 100, 110, 120, 130, 150],
             'max_features':['auto', 'sqrt', 'log2', None],
             'random_state':[0]
            }

model = grid(model=DecisionTreeClassifier(), params=tree_para, n_jobs=n_jobs)

Mejor conjunto de parámetros:
{'criterion': 'entropy', 'max_depth': None, 'max_features': None, 'random_state': 0}

Puntajes de la grilla:

Exactitud: 0.6918 (+/-0.0411) para los parámetros {'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'random_state': 0}
Exactitud: 0.6918 (+/-0.0411) para los parámetros {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'random_state': 0}
Exactitud: 0.6918 (+/-0.0411) para los parámetros {'criterion': 'gini', 'max_depth': None, 'max_features': 'log2', 'random_state': 0}
Exactitud: 0.7005 (+/-0.0659) para los parámetros {'criterion': 'gini', 'max_depth': None, 'max_features': None, 'random_state': 0}
Exactitud: 0.5959 (+/-0.0372) para los parámetros {'criterion': 'gini', 'max_depth': 4, 'max_features': 'auto', 'random_state': 0}
Exactitud: 0.5959 (+/-0.0372) para los parámetros {'criterion': 'gini', 'max_depth': 4, 'max_features': 'sqrt', 'random_state': 0}
Exactitud: 0.5959 (+/-0.0372) para los parámetros {'criterion': 

Corremos el modelo con los mejores parámetros encontrados:

In [29]:
best_params = model.best_params_

model = DecisionTreeClassifier(criterion=best_params['criterion'], 
                               max_depth=best_params['max_depth'],
                               max_features=best_params['max_features'],
                               random_state=best_params['random_state']
                              )
model.fit(xTrain, yTrain)

yPred = model.predict(xTest)

desc = "criterion={}, max_depth={}, max_features={}, random_state={}"
desc = desc.format(str(best_params['criterion']),
                   str(best_params['max_depth']),
                   str(best_params['max_features']), 
                   str(best_params['random_state'])
                  )

save(num_ejec=i, y_pred=yPred, tipo_modelo='DecisionTreeClassifier', descripcion=desc)
i+=1

Reporte para el modelo número 3 :
              precision    recall  f1-score   support

           1       0.19      0.18      0.19       288
           2       0.58      0.66      0.62      2012
           3       0.49      0.47      0.48      1658
           4       0.55      0.50      0.53      1143
           5       0.47      0.47      0.47       561
           6       0.46      0.45      0.45       285
           7       0.65      0.60      0.63       929

    accuracy                           0.54      6876
   macro avg       0.49      0.48      0.48      6876
weighted avg       0.53      0.54      0.53      6876

Actualización de la tabla:
  id_ejecución             tipo_modelo  \
0        ejec0      LogisticRegression   
1        ejec1      LogisticRegression   
2        ejec2  DecisionTreeClassifier   
3        ejec3  DecisionTreeClassifier   

                                         descripcion  accuracy  precision  \
0  default parameters (penalty=l2, C=1.0, random_...  

4- Entrenar un modelo Random Forest

In [10]:
# Random Forest
model = RandomForestClassifier(random_state=0) 
model.fit(xTrain, yTrain)

yPred = model.predict(xTest)

desc = 'default parameters (n_estimators=10, criterion=gini, max_depth=None, max_features=auto, random_state=0)'

save(num_ejec=i, y_pred=yPred, tipo_modelo='RandomForestClassifier', descripcion=desc) 
i+=1

/home/nahuel/anaconda3/envs/diploDatos/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Reporte para el modelo número 1 :
              precision    recall  f1-score   support

           1       0.29      0.18      0.22       288
           2       0.57      0.68      0.62      2012
           3       0.47      0.48      0.48      1658
           4       0.56      0.51      0.53      1143
           5       0.53      0.45      0.49       561
           6       0.54      0.42      0.47       285
           7       0.69      0.63      0.66       929

    accuracy                           0.55      6876
   macro avg       0.52      0.48      0.50      6876
weighted avg       0.54      0.55      0.54      6876

Actualización de la tabla:
  id_ejecución             tipo_modelo  \
0        ejec0      LogisticRegression   
1        ejec1  RandomForestClassifier   

                                         descripcion  accuracy  precision  \
0  default parameters (penalty=l2, C=1.0, random_...  0.386271   0.291249   
1  default parameters (n_estimators=10, criterion...  0.54653

Buscamos mejores parámetros:

In [10]:
rand_para = {'n_estimators':[5, 10, 15, 20, 25, 30, 35, 40, 50, 60, 10, 80, 90, 100, 200, 300, 320, 350, 400, 450],
             'criterion':['gini', 'entropy'],
             'max_depth':[None, 8, 9, 10, 11, 12, 15, 20, 30, 40, 50, 70, 90, 100, 110, 120, 130, 150, 200, 250],
             'max_features':['sqrt', 'log2', None],
             'random_state': [0],
            }

model = grid(model=RandomForestClassifier(), params=rand_para, n_jobs=n_jobs, verbose=10)

Fitting 3 folds for each of 3200 candidates, totalling 9600 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    2.6s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    4.5s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:    8.0s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   13.0s
/home/nahuel/anaconda3/envs/diploDatos/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


KeyboardInterrupt: 

(La celda anterior tardó aproximadamente un día en ejecutarse).

Corremos el modelo con los mejores parámetros encontrados:

In [ ]:
best_params = model.best_params_

model = RandomForestClassifier(n_estimators=best_params['n_estimators'],
                               criterion=best_params['criterion'],
                               max_depth=best_params['max_depth'],
                               max_features=best_params['max_features'],
                               random_state=best_params['random_state']
                              )
model.fit(xTrain, yTrain)

yPred = model.predict(xTest)

desc = "n_estimators={}, criterion={}, max_depth={}, max_features={}, random_state={}"
desc = desc.format(str(best_params['n_estimators']),
                   str(best_params['criterion']),
                   str(best_params['max_depth']), 
                   str(best_params['max_features']),
                   str(best_params['random_state'])
                  )
                
save(num_ejec=i, y_pred=yPred, tipo_modelo='RandomForestClassifier', descripcion=desc)
i+=1

5- Entrenar un modelo con XGBoost
(Ejemplo: https://www.kaggle.com/rozester/xgboost-example-python)

In [ ]:
# XGBoost 
model = xgb.XGBClassifier(random_state=0)
model.fit(xTrain, yTrain)

yPred = model.predict(xTest)

desc = 'default parameters (max_depth=3, learning_rate=0.1, n_estimators=100, random_state=0)'

save(num_ejec=i, y_pred=yPred, tipo_modelo='XGBClassifier', descripcion=desc) 
i+=1

Buscamos mejores parámetros:

In [ ]:
xgb_para = {'n_estimators':[5, 10, 15, 20, 25, 30, 35, 40, 50, 60, 10, 80, 90, 100, 200, 300, 320, 340, 350, 400],
            'max_depth':[3, 6, 8, 9, 10, 11, 12, 15, 20, 30, 40, 50, 70, 90, 100, 110, 120, 130, 150, 200, 300],
            'learning_rate':[0.01, 0.05, 0.1],
            'random_state':[0]
           }

model = grid(model=xgb.XGBClassifier(), params=xgb_para, n_jobs=n_jobs)

(la anterior celda tarda días en ejecutarse, habría que enviarla al cluster)

Corremos el modelo con los mejores parámetros encontrados:

In [ ]:
best_params = model.best_params_

model = xgb.XGBClassifier(n_estimators=best_params['n_estimators'], 
                          max_depth=best_params['max_depth'],
                          learning_rate=best_params['learning_rate'],
                          random_state=best_params['random_state']
                         )
model.fit(xTrain, yTrain)

yPred = model.predict(xTest)

desc = "n_estimators={}, max_depth={}, learning_rate={}, random_state={}"
desc = desc.format(str(best_params['n_estimators']),
                str(best_params['max_depth']), 
                str(best_params['learning_rate']),
                str(best_params['random_state'])
                )

save(num_ejec=i, y_pred=yPred, tipo_modelo='XGBClassifier', descripcion=desc)
i+=1